In [1]:
import pandas as pd
from sklearn import  linear_model
import numpy as np
import matplotlib.pyplot as plt
import math

In [ ]:
sales = pd.read_csv("kc_house_data.csv")

In [43]:
sales.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900,3,1.00,1180,5650,1,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000,3,2.25,2570,7242,2,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000,2,1.00,770,10000,1,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000,4,3.00,1960,5000,1,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000,3,2.00,1680,8080,1,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [58]:
#Add 4 new variables in both your train_data and test_data. 
sales["bedrooms_squared"] = sales["bedrooms"]*sales["bedrooms"]
sales["bed_bath_rooms"] = sales["bedrooms"]*sales["bathrooms"]
sales["log_sqft_living"] = np.log(sales["sqft_living"])
sales["lat_plus_long"] = sales["lat"] + sales["long"]

In [63]:
train_data = sales[:-20]

In [144]:
test_data = pd.read_csv("kc_house_test_data.csv")

In [147]:
test_data["bedrooms_squared"] = test_data["bedrooms"]*test_data["bedrooms"]
test_data["bed_bath_rooms"] = test_data["bedrooms"]*test_data["bathrooms"]
test_data["log_sqft_living"] = np.log(test_data["sqft_living"])
test_data["lat_plus_long"] = test_data["lat"] + test_data["long"]

In [140]:
train_data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,bedrooms_squared,bed_bath_rooms,log_sqft_living,lat_plus_long
0,7129300520,20141013T000000,221900,3,1.00,1180,5650,1,0,0,...,0,98178,47.5112,-122.257,1340,5650,9,3.00,7.073270,-74.7458
1,6414100192,20141209T000000,538000,3,2.25,2570,7242,2,0,0,...,1991,98125,47.7210,-122.319,1690,7639,9,6.75,7.851661,-74.5980
2,5631500400,20150225T000000,180000,2,1.00,770,10000,1,0,0,...,0,98028,47.7379,-122.233,2720,8062,4,2.00,6.646391,-74.4951
3,2487200875,20141209T000000,604000,4,3.00,1960,5000,1,0,0,...,0,98136,47.5208,-122.393,1360,5000,16,12.00,7.580700,-74.8722
4,1954400510,20150218T000000,510000,3,2.00,1680,8080,1,0,0,...,0,98074,47.6168,-122.045,1800,7503,9,6.00,7.426549,-74.4282


In [148]:
test_data.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,bedrooms_squared,bed_bath_rooms,log_sqft_living,lat_plus_long
count,4.229000e+03,4229.000000,4229.000000,4229.000000,4229.000000,4229.000000,4229.000000,4229.000000,4229.000000,4229.000000,...,4229.000000,4229.000000,4229.000000,4229.000000,4229.000000,4229.000000,4229.000000,4229.000000,4229.000000,4229.000000
mean,4.604771e+09,543054.043036,3.376921,2.113561,2079.366280,15168.859068,1.494561,0.007094,0.227004,3.403878,...,89.722629,98077.951762,47.563092,-122.216426,1988.844171,12735.877749,12.446678,7.503902,7.550275,-74.653334
std,2.894017e+09,356421.245803,1.021434,0.763560,905.317454,41265.627014,0.542292,0.083936,0.759375,0.654686,...,413.736867,53.426623,0.137965,0.140497,680.837632,27829.200218,17.868973,4.227032,0.426554,0.182162
min,1.000102e+06,85000.000000,0.000000,0.000000,370.000000,600.000000,1.000000,0.000000,0.000000,1.000000,...,0.000000,98001.000000,47.155900,-122.514000,700.000000,660.000000,0.000000,0.000000,5.913503,-75.177200
25%,2.110900e+09,325000.000000,3.000000,1.750000,1430.000000,5027.000000,1.000000,0.000000,0.000000,3.000000,...,0.000000,98032.000000,47.476600,-122.330000,1490.000000,5105.000000,9.000000,4.500000,7.265430,-74.767200
50%,4.019301e+09,453000.000000,3.000000,2.250000,1920.000000,7633.000000,1.500000,0.000000,0.000000,3.000000,...,0.000000,98065.000000,47.573400,-122.239000,1840.000000,7611.000000,9.000000,7.000000,7.560080,-74.667100
75%,7.338220e+09,650000.000000,4.000000,2.500000,2550.000000,10760.000000,2.000000,0.000000,0.000000,4.000000,...,0.000000,98118.000000,47.679500,-122.125000,2370.000000,10159.000000,16.000000,10.000000,7.843849,-74.528100
max,9.895000e+09,6885000.000000,33.000000,7.750000,9890.000000,1024068.000000,3.500000,1.000000,4.000000,5.000000,...,2015.000000,98199.000000,47.777600,-121.315000,5030.000000,858132.000000,1089.000000,57.750000,9.199279,-73.602000


In [68]:
predictors_Model1 = ["sqft_living", "bedrooms", "bathrooms", "lat", "long"]

In [70]:
alg = linear_model.LinearRegression()

In [71]:
train_predictors_Model1 = train_data[predictors_Model1 ]

In [82]:
train_target = train_data["price"]

In [84]:
alg.fit(train_predictors_Model1, train_target)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [85]:
print('Coefficients: %x', alg.coef_)

('Coefficients: %x', array([  3.07689052e+02,  -5.35985181e+04,   1.66163768e+04,
         6.57726739e+05,  -3.12360920e+05]))


In [86]:
print("Residual sum of squares: %.2f"
      % np.mean((alg.predict(train_predictors_Model1) - train_target) ** 2))

Residual sum of squares: 55210364235.65


In [93]:
predictors_Model2 = [ "sqft_living", "bedrooms", "bathrooms", "lat","long", "bed_bath_rooms"]

In [94]:
train_predictors_Model2 = train_data[predictors_Model2 ]

In [98]:
alg.fit(train_predictors_Model2, train_target)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [99]:
print('Coefficients: %x', alg.coef_)

('Coefficients: %x', array([  3.01765621e+02,  -1.06874528e+05,  -7.11934306e+04,
         6.54152686e+05,  -2.97121565e+05,   2.57299198e+04]))


In [101]:
print("Residual sum of squares: %.2f"
      % np.mean((alg.predict(train_predictors_Model2) - train_target) ** 2))

Residual sum of squares: 54668922342.44


In [172]:
predictors_Model3 = ["sqft_living", "bedrooms", "bathrooms", "lat","long", "bed_bath_rooms", "bedrooms_squared", "log_sqft_living", "lat_plus_long"]

In [173]:
train_predictors_Model3 = train_data[predictors_Model3]

In [174]:
alg.fit(train_predictors_Model3, train_target)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [175]:
print('Coefficients: %x', alg.coef_)

('Coefficients: %x', array([  5.22223434e+02,  -5.82373535e+03,   9.04298460e+04,
         5.29937457e+05,  -4.04634999e+05,  -1.49129833e+04,
         7.60741342e+02,  -5.43907658e+05,   1.25302458e+05]))


In [176]:
print("Residual sum of squares: %.2f"
      % np.mean((alg.predict(train_predictors_Model3) - train_target) ** 2))

Residual sum of squares: 51703166802.44


In [161]:
test_predictors_Model1 = test_data[predictors_Model1]

In [162]:
test_target = test_data["price"]

In [163]:
alg.fit(test_predictors_Model1, test_target)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [164]:
print('Coefficients: %x', alg.coef_)

('Coefficients: %x', array([  2.91523046e+02,  -3.63442805e+04,   2.24784634e+04,
         6.49781148e+05,  -3.22967664e+05]))


In [165]:
print("Residual sum of squares: %.2f"
      % np.mean((alg.predict(test_predictors_Model1) - test_target) ** 2))

Residual sum of squares: 52870293345.51


In [179]:
test_predictors_Model2 = test_data[predictors_Model2]

In [180]:
alg.fit(test_predictors_Model2, test_target)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [181]:
print("Residual sum of squares: %.2f"
      % np.mean((alg.predict(test_predictors_Model2) - test_target) ** 2))

Residual sum of squares: 52231801236.18


In [169]:
test_predictors_Model3 = test_data[predictors_Model3]

In [170]:
alg.fit(test_predictors_Model3, test_target)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [177]:
print("Residual sum of squares: %.2f"
      % np.mean((alg.predict(test_predictors_Model3) - test_target) ** 2))

Residual sum of squares: 50182086123.15
